Antes de mais nada os notebooks aqui mostrado tiveram como base/foram retirados dos seguintes repositórios: 
 > https://github.com/fchollet/deep-learning-with-python-notebooks 
 
 
 > https://github.com/cdfmlr/Deep-Learning-with-Python-Notebooks
 
 Sugiro fortemente que consultem os códigos originais e em caso de dúvida podem me contatar para conversarmos. 

# Aprendizado profundo com Python

## 7.1 Indo além do modelo Sequencial: a API funcional Keras

> Solução sem modelo sequencial: API funcional Keras
    
O modelo sequencial que usamos antes é o modelo mais básico, mas comumente usado. Ele tem apenas uma entrada e uma saída, e toda a rede é formada por empilhamento linear de camadas.

No entanto, às vezes nossa rede requer várias entradas. Por exemplo, para prever o preço das roupas, insira as informações do produto, a descrição do texto e as imagens. Esses três tipos de informações devem ser processados pela Dense, RNN e CNN respectivamente. 


Às vezes, nossa rede requer várias saídas (vários cabeçalhos). Por exemplo, insira um romance, esperamos obter a classificação do romance e adivinhar o tempo de escrita. Este problema deve usar um módulo comum para processar o texto, extrair as informações e, em seguida, submetê-lo ao novo classificador e regressor de data para prever a classificação e o tempo de escrita.

Às vezes, algumas redes complexas usam topologias de rede não lineares. Por exemplo, uma coisa chamada Iniciação, a entrada será processada por vários ramos de convolução paralela e, em seguida, as saídas desses ramos são mescladas em um único tensor; há também um método chamado conexão residual (conexão residual), a saída anterior O tensor é adicionado ao tensor de saída subsequente para reinjetar a representação anterior no fluxo de dados downstream para evitar a perda de informações no processo de processamento de informações.

Essas redes são semelhantes a gráficos, uma estrutura de rede, ao invés de uma pilha linear como Sequencial. Para implementar este modelo complexo no Keras, você precisa usar a API funcional do Keras.

### API Funcional

A API funcional de Keras usa camadas como funções, recebe tensores e retorna tensores:

In [1]:
Antes de mais nada os notebooks aqui mostrado tiveram como base/foram retirados dos seguintes repositórios: 
 > https://github.com/fchollet/deep-learning-with-python-notebooks 
 
 
 > https://github.com/cdfmlr/Deep-Learning-with-Python-Notebooks
 
 Sugiro fortemente que consultem os códigos originais e em caso de dúvida podem me contatar para conversarmos. from tensorflow.keras import Input, layers

input_tensor = Input(shape=(32, ))    # Tensor de entrada
dense = layers.Dense(32, activation='relu')    # Função de camada
output_tensor = dense(input_tensor)   # Tensor de saída 

Vamos construir uma rede simples usando API funcional e comparar com Sequential:

In [2]:
# Sequential modelo

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64, )))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

seq_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [3]:
# Modelo de API funcional

from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras import layers

input_tensor = Input(shape=(64, ))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

func_model = Model(input_tensor, output_tensor)

func_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


Quando o objeto Model é instanciado, apenas o tensor de entrada e o tensor de saída obtidos pela transformação do tensor de entrada (através de várias camadas) são fornecidos. Keras encontrará automaticamente cada camada incluída de input_tensor a output_tensor e combinará essas camadas em uma estrutura de dados semelhante a um gráfico - um modelo.

Observe que output_tensor deve ser transformado do input_tensor correspondente. Se você usar tensores de entrada e tensores de saída não relacionados para construir o modelo, ValueError desconectado do Graph será explodido (as keras escritas no livro são RuntimeError, tf.keras é ValueError):

`` `python
>>> unrelated_input = Input (shape = (32,))
>>> bad_model = Model (unrelated_input, output_tensor)
... # Traceback
ValueError: Gráfico desconectado: não é possível obter o valor do tensor Tensor ("input_2: 0", shape = (None, 64), dtype = float32) na camada "dense_4". As seguintes camadas anteriores foram acessadas sem problemas: []
`` `

Em outras palavras, é impossível conectar da entrada especificada à saída para formar um gráfico (Gráfico, o tipo de estrutura de dados, o tipo de rede).

Para a rede construída por esta API funcional, a compilação, treinamento ou avaliação são iguais a Sequencial.

In [7]:
# Compilar
func_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# Dados de treinamento aleatórios
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

# Treinamento
func_model.fit(x_train, y_train, epochs=10, batch_size=128)

# Avaliação
score = func_model.evaluate(x_train, y_train)

Epoch 1/10
8/8 [==============================] - 0s 1ms/step - loss: 33.5245
Epoch 2/10
8/8 [==============================] - 0s 1ms/step - loss: 38.1499
Epoch 3/10
8/8 [==============================] - 0s 2ms/step - loss: 42.0662
Epoch 4/10
8/8 [==============================] - 0s 1ms/step - loss: 46.1707
Epoch 5/10
8/8 [==============================] - 0s 813us/step - loss: 50.6095
Epoch 6/10
8/8 [==============================] - 0s 1ms/step - loss: 54.8778
Epoch 7/10
8/8 [==============================] - 0s 1ms/step - loss: 59.6547
Epoch 8/10
8/8 [==============================] - 0s 1ms/step - loss: 64.5255
Epoch 9/10
8/8 [==============================] - 0s 1ms/step - loss: 69.2659
Epoch 10/10
32/32 [==============================] - 0s 617us/step - loss: 78.1296


### Modelo de múltiplas entradas

APIs funcionais podem ser usadas para construir modelos com várias entradas. Modelos com várias entradas geralmente combinam diferentes ramificações com uma camada que pode combinar tensores em um determinado ponto. Para combinar tensores, você pode usar adição, concatenação, etc. Camadas como `keras.layers.add` e` keras.layers.concatenate` são fornecidas no keras para completar essas operações.

Veja um exemplo específico e faça um modelo de perguntas e respostas. Um modelo típico de resposta a perguntas usa duas entradas:

- Texto da pergunta
- Fornece texto informativo para responder a perguntas (como artigos de notícias relacionados)

O modelo precisa gerar (produzir) uma resposta. O caso mais simples é responder apenas uma palavra.Podemos obter o resultado enfrentando algum vocabulário predefinido e tornando a saída softmax.

Para implementar este modelo com API funcional, primeiro construímos dois ramos independentes, representamos o texto de referência e a pergunta como vetores respectivamente, em seguida, conectamos esses dois vetores e adicionamos um classificador softmax à representação após a conexão ser concluída:

In [29]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# referência
text_input = Input(shape=(None, ), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

# problema
question_input = Input(shape=(None, ), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# Mesclando referências, ramificações com problemas
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# Classificador de nível superior
answer = layers.Dense(anser_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer, name='QA')

model.summary()

model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy', 
              metrics=['acc'])

Model: "QA"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, None, 32)     320000      question[0][0]                   
_________________________________________________________________________________________________

Ao treinar este modelo de múltiplas entradas, você pode transferi-lo para uma lista de componentes e também pode transferir um dicionário para a entrada com um nome especificado:

In [31]:
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, 
                         size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, 
                             size=(num_samples, max_length))
answers = np.random.randint(0, 1, 
                            size=(num_samples, answer_vocabulary_size)) # one-hot Codificado

# Método 1. Lista de upload
model.fit([text, question], answers, epochs=2, batch_size=128)

# Método 2. Transferir um dicionário
model.fit({'text': text, 'question': question}, answers, epochs=2, batch_size=128)

Epoch 1/2
8/8 [==============================] - 0s 54ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 2/2
8/8 [==============================] - 0s 57ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 1/2
8/8 [==============================] - 0s 53ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 2/2
8/8 [==============================] - 0s 54ms/step - loss: 0.0000e+00 - acc: 0.0000e+00


### Modelo de múltiplas saídas

Também é conveniente usar APIs funcionais para construir modelos com várias saídas (multi-heads). Por exemplo, vamos fazer uma rede que tenta prever a natureza diferente dos dados ao mesmo tempo: insira algumas postagens de mídia social de alguém e tente prever os atributos de idade, sexo e nível de renda dessa pessoa:

A implementação específica é muito simples, basta escrever 3 saídas diferentes no final:

In [32]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dense
from tensorflow.keras import Input
from tensorflow.keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_post = layers.Embedding(256, vocabulary_size)(posts_input)
x = Conv1D(128, 5, activation='relu')(embedded_post)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation="relu")(x)
x = Conv1D(256, 5, activation="relu")(x)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation="relu")(x)
x = Conv1D(256, 5, activation="relu")(x)
x = GlobalMaxPool1D()(x)
x = Dense(128, activation='relu')(x)

# Defina vários cabeçalhos (saída)
age_prediction = Dense(1, name='age')(x)
income_prediction = Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

model.summary()

Model: "functional_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, None, 50000)  12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 128)    32000128    embedding_15[0][0]               
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, None, 128)    0           conv1d[0][0]                     
_______________________________________________________________________________________

** Compilação de modelo de várias cabeças **

Ao compilar este modelo, porque existem objetivos diferentes, deve-se observar que diferentes funções de perda precisam ser especificadas para cada cabeça da rede.

Mas o papel da descida gradiente só pode ser minimizar um escalar, então em Keras, as diferentes perdas especificadas para diferentes saídas em tempo de compilação serão adicionadas para obter uma perda global durante o treinamento. O processo de treinamento é para minimizar esta perda global 化.

Nesse caso, se a contribuição da perda for gravemente desequilibrada, o modelo priorizará a tarefa com o maior valor de perda, sem considerar as outras tarefas. Para resolver este problema, diferentes perdas podem ser ponderadas. Por exemplo, o valor de perda de mse é geralmente 3 ~ 5 e o valor de perda de binary_crossentropy é geralmente tão baixo quanto 0,1. Para equilibrar a contribuição de perda, podemos fazer o peso de binary_crossentropy 10 e o peso de perda de mse 0,5.

A atribuição de múltiplas perdas e pesos é feita usando listas ou dicionários:
```python
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

# Ou se você nomear a camada de saída, pode usar um dicionário:
model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})
```

** Treinamento do modelo multi-head **

Ao treinar este modelo, basta passar a saída de destino em uma lista ou dicionário:

```python
model.fit(posts, [age_targets, income_targets, gender_targets],
          epochs=10, batch_size=64)

# or

model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
          epochs=10, batch_size=64)
```

### Gráfico Acíclico Direcionado em Camadas

Usando APIs funcionais, além de construir modelos de múltiplas entradas e saídas, também podemos implementar redes com topologias internas complexas.

Na verdade, a rede neural em Keras pode ser qualquer gráfico acíclico direcionado composto de camadas. Os componentes da estrutura gráfica mais conhecidos incluem o módulo de iniciação e a conexão residual.

#### Módulo de iniciação

A iniciação é uma pilha de módulos, cada um parecendo uma pequena rede independente, esses módulos são divididos em vários ramos paralelos e, finalmente, os recursos resultantes são conectados entre si. Essa operação permite que a rede aprenda os recursos espaciais e os recursos canal a canal da imagem separadamente, o que é mais eficaz do que usar uma rede para aprender esses recursos ao mesmo tempo.

> Observação: a convolução 1x1 usada aqui é chamada de convolução pontual, que é um recurso do módulo de Iniciação.
> Ele olha apenas para um pixel por vez, e os recursos calculados podem misturar as informações nos canais de entrada, mas não as informações espaciais.
> Desta forma, é feita uma distinção entre aprendizagem de recursos de canal e aprendizagem de recursos espaciais.

Isso pode ser alcançado com uma API funcional:

In [41]:
from tensorflow.keras import layers

x = Input(shape=(None, None, 3))    # Imagem RGB

branch_a = layers.Conv2D(128, 1, activation='relu', strides=2, padding='same')(x)

branch_b = layers.Conv2D(128, 1, activation='relu', padding='same')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2, padding='same')(x)
branch_c = layers.Conv2D(128, 3, activation='relu', padding='same')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu', padding='same')(x)
branch_d = layers.Conv2D(128, 3, activation='relu', padding='same')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(branch_d)

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

Na verdade, Keras tem uma arquitetura Inception V3 completa integrada. Ele pode ser chamado por `keras.applications.inception_v3.InceptionV3`.

Relacionado ao Inception, existe outra coisa chamada ** Xception **. A palavra Xception significa início extremo. É um tipo de início extremo, que separa completamente o aprendizado de recursos de canal do aprendizado de recursos espaciais. O número de parâmetros do Xception é aproximadamente o mesmo do Inception V3, mas seu uso de parâmetros é mais eficiente, portanto, tem melhor desempenho e maior precisão em conjuntos de dados de grande escala.

#### Conexão residual

A conexão residual (conexão residual) é um componente muito comumente usado agora, ele resolve o problema de desaparecimento e gargalo do gradiente do modelo de aprendizagem profunda em larga escala. Em geral, adicionar conexões residuais a qualquer modelo com mais de 10 camadas pode ajudar.

- Desaparecimento de radiação: há mais camadas passadas e a representação aprendida anteriormente torna-se borrada ou mesmo completamente perdida, fazendo com que a rede falhe no treinamento.
-Indica um gargalo: empilhar camadas, a última camada só pode acessar as coisas aprendidas na camada anterior. Se uma determinada camada for muito pequena (menos informações podem ser inseridas na ativação), as informações serão cardadas e um gargalo aparecerá.

A conexão residual é permitir a saída de uma camada anterior como a entrada de uma camada posterior (criando um atalho na rede). A saída da camada anterior não está conectada com a ativação da última camada, mas é adicionada à ativação da última camada (se a forma for diferente, use uma transformação linear para alterar a ativação da camada anterior na forma de destino).

> Observação: a transformação linear pode usar a camada Densa sem ativação ou usar a convolução 1 × 1 sem ativação no CNN.

```python
from keras import layers

x = ...

y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

# Formas diferentes, faça transformação linear:
residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)  # Use a convolução 1 × 1 para reduzir linearmente x para ter a mesma forma que y

y = layers.add([y, residual])
```

#### Peso da camada compartilhada

Usando APIs funcionais, outra operação é usar uma instância de camada várias vezes. Se você chamar a mesma instância de camada várias vezes, poderá reutilizar o mesmo peso. Usando este recurso, um modelo com ramificações compartilhadas pode ser construído, ou seja, várias ramificações compartilham o mesmo conhecimento e realizam as mesmas operações.

Por exemplo, queremos avaliar a semelhança semântica entre duas frases. Este modelo usa duas frases como entrada e produz uma pontuação que varia de 0 a 1. Quanto maior o valor, mais semelhante é o significado da frase.

Neste problema, as duas sentenças de entrada são intercambiáveis ​​(a similaridade da sentença A com B é igual à similaridade de B com A). Portanto, dois modelos separados não devem ser aprendidos a processar duas sentenças de entrada separadamente. Em vez disso, uma camada LSTM deve ser usada para processar duas frases. A representação (peso) desta camada LSTM é aprendida de duas entradas ao mesmo tempo. Este modelo é denominado Siamese LSTM (Siamese LSTM) ou LSTM compartilhado (LSTM compartilhado).

Esse modelo é implementado usando o compartilhamento de camadas na API funcional Keras:

In [42]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model

lstm = layers.LSTM(32)  # Instancie apenas um LSTM

left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)

model.summary()

Model: "functional_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
lstm_15 (LSTM)                  (None, 32)           20608       input_13[0][0]                   
                                                                 input_14[0][0]                   
__________________________________________________________________________________________________
concatenate_13 (Concatenate)    (None, 64)           0           lstm_15[0][0]         

#### Use o modelo como uma camada

No Keras, podemos usar o modelo como uma camada (o fenômeno do modelo é uma camada grande) e as classes Sequential e Model podem ser usadas como camadas. Basta chamá-lo funcionalmente como uma camada:

`` `python
y = modelo (x)
y1, y2 = model_with_multi_inputs_and_outputs ([x1, x2])
`` `

Por exemplo, processamos um modelo visual com câmeras duplas como entrada (este modelo pode perceber a profundidade). Usamos o modelo applications.Xception como a camada e usamos o método de camada compartilhada anterior para implementar esta rede:

In [46]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras import applications

xception_base = applications.Xception(weights=None, include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_input = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_input], axis=-1)
